In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
pd.options.display.float_format = '{:.6f}'.format

# 데이터 불러오기

In [26]:
train = pd.read_csv('./Dataset/train.csv')
test = pd.read_csv('./Dataset/test.csv')
car_2020 = pd.read_csv('./Dataset/car_2020.csv',encoding='cp949')
submission = pd.read_csv('./Dataset/sample_submission.csv')

In [27]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.720000,134,38.000000,A,15667000,103680,0.000000,3.000000,1425.000000,1015.000000
1,C2483,900,아파트,경상북도,국민임대,39.720000,15,38.000000,A,15667000,103680,0.000000,3.000000,1425.000000,1015.000000
2,C2483,900,아파트,경상북도,국민임대,51.930000,385,38.000000,A,27304000,184330,0.000000,3.000000,1425.000000,1015.000000
3,C2483,900,아파트,경상북도,국민임대,51.930000,15,38.000000,A,27304000,184330,0.000000,3.000000,1425.000000,1015.000000
4,C2483,900,아파트,경상북도,국민임대,51.930000,41,38.000000,A,27304000,184330,0.000000,3.000000,1425.000000,1015.000000


car_2020 = car_2020.drop(['시군구(1)'], axis=1)
car_2020.columns = ['지역','월','항목','분류','자동차수']
car_2020['월'] = pd.to_datetime(car_2020['월']).dt.month
car_2020.head()

In [28]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

# 컬렴명 바꿔주기

In [29]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]


# 지하철 Y or N로 변경

In [30]:
train['지하철'] = train['지하철'].fillna(0)
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)

# 결측치 처리

In [31]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

In [32]:
train['버스'] = train['버스'].fillna(train['버스'].mean())

In [33]:
train.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

# 임대건물구분 : 아파트->상가 / 전용면적별세대수 : x->1

In [54]:
idx = train[(train['임대건물구분']=='아파트') & (train['신분']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

In [56]:
train[(train['임대건물구분']=='상가') & (train['신분']=='D')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
88,C1925,601,상가,5,임대상가,30.000000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
89,C1925,601,상가,5,임대상가,30.000000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
90,C1925,601,상가,5,임대상가,30.000000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
91,C1925,601,상가,5,임대상가,70.000000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
101,C1874,619,상가,7,임대상가,15.000000,1,2.000000,D,0.000000,0.000000,0,2.000000,97.000000,62.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,C1350,1401,상가,2,공공분양,70.000000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2334,C1350,1401,상가,2,공공분양,80.000000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2335,C1350,1401,상가,2,공공분양,80.000000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2336,C1350,1401,상가,2,공공분양,80.000000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000


In [10]:
# 원핫 인코딩
#train = pd.get_dummies(data = train, columns = ['지하철'])

# 지역명 숫자로 매핑

In [57]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

# 전용면적을 5의 배수로 변경

In [58]:
train['전용면적'] = train['전용면적']//5*5

# 전용면적 상/하한 적용

In [59]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2952 non-null   object 
 1   총세대수      2952 non-null   int64  
 2   임대건물구분    2952 non-null   object 
 3   지역        2952 non-null   int64  
 4   공급유형      2952 non-null   object 
 5   전용면적      2952 non-null   float64
 6   전용면적별세대수  2952 non-null   int64  
 7   공가수       2952 non-null   float64
 8   신분        2952 non-null   object 
 9   임대보증금     2952 non-null   float64
 10  임대료       2952 non-null   float64
 11  지하철       2952 non-null   int32  
 12  버스        2952 non-null   float64
 13  단지내주차면수   2952 non-null   float64
 14  등록차량수     2952 non-null   float64
dtypes: float64(7), int32(1), int64(3), object(4)
memory usage: 334.5+ KB


# test 불러오기

In [60]:
test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [61]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [62]:
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)

In [63]:
test['지하철'] = test['지하철'].fillna(test['지하철'].mean())

In [64]:
test.loc[test.단지코드.isin(['C2411']) & test.신분.isnull(), '신분'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.신분.isnull(), '신분'] = 'C'

In [68]:
local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

In [65]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [66]:
columns = ['총세대수', '지역', '전용면적별세대수', '공가수', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [44]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [45]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']

for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:04, 92.87it/s]
150it [00:01, 83.54it/s]


In [24]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [28]:
forest = RandomForestRegressor(n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [29]:
pred = forest.predict(x_test)
#pred

In [30]:
submission['num'] = pred

In [31]:
submission.to_csv('baseline.csv', index=False)